In [2]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import json
import requests
import pymongo

In [3]:
# Connect to database
conn = sqlite3.connect('''database.sqlite''')

# Create cursor object
cur = conn.cursor()

In [4]:
# View Matches dataframe

cur.execute('''SELECT * FROM Matches;''')
Matches_df =pd.DataFrame(cur.fetchall())
Matches_df.columns = [x[0] for x in cur.description]
Matches_df.head()

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,1,D2,2009,2010-04-04,Oberhausen,Kaiserslautern,2,1,H
1,2,D2,2009,2009-11-01,Munich 1860,Kaiserslautern,0,1,A
2,3,D2,2009,2009-10-04,Frankfurt FSV,Kaiserslautern,1,1,D
3,4,D2,2009,2010-02-21,Frankfurt FSV,Karlsruhe,2,1,H
4,5,D2,2009,2009-12-06,Ahlen,Karlsruhe,1,3,A


In [5]:
# View Teams dataframe

cur.execute('''SELECT * FROM Teams;''')
Teams_df = pd.DataFrame(cur.fetchall())
Teams_df.columns = [x[0] for x in cur.description]
Teams_df.head()

,Season,TeamName,KaderHome,AvgAgeHome,ForeignPlayersHome,OverallMarketValueHome,AvgMarketValueHome,StadiumCapacity
0,2017,Bayern Munich,27,26,15,597950000,22150000,75000
1,2017,Dortmund,33,25,18,416730000,12630000,81359
2,2017,Leverkusen,31,24,15,222600000,7180000,30210
3,2017,RB Leipzig,30,23,15,180130000,6000000,42959
4,2017,Schalke 04,29,24,17,179550000,6190000,62271


In [6]:
# View Teams_in_Matches dataframe

cur.execute('''SELECT * FROM Teams_in_Matches;''')
Teams_in_Matches_df = pd.DataFrame(cur.fetchall())
Teams_in_Matches_df.columns = [x[0] for x in cur.description]
Teams_in_Matches_df.head()

,Match_ID,Unique_Team_ID
0,1,26
1,1,46
2,2,26
3,2,42
4,3,26


In [7]:
# View Unique_Teams dataframe

cur.execute('''SELECT * FROM Unique_Teams;''')
Unique_Teams_df = pd.DataFrame(cur.fetchall())
Unique_Teams_df.columns = [x[0] for x in cur.description]
Unique_Teams_df.head()

,TeamName,Unique_Team_ID
0,Bayern Munich,1
1,Dortmund,2
2,Leverkusen,3
3,RB Leipzig,4
4,Schalke 04,5


In [8]:
# View Matches from 2011 season

cur.execute('''SELECT * FROM Matches WHERE Season = '2011';''')
Matches_2011_df =pd.DataFrame(cur.fetchall())
Matches_2011_df.columns = [x[0] for x in cur.description]
Matches_2011_df.head()

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A
1,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A
2,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A
3,1095,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H
4,1096,D1,2011,2012-02-18,Freiburg,Bayern Munich,0,0,D


In [9]:
Matches_2011_df.Date.nunique()

165

# DarkSky API WeatherGetter

The below class will take in a list of dates and get the weather in Berlin on that date. It will also produce each teams percentage of wins that occured in the rain.

In [10]:
class WeatherGetter:
    
    def __init__(self):
        self.path = "/Users/erikadauria/Flatiron/Projects/Soccer_in_the_Rain/.secret/darksky_api.json"
        self.url = 'https://api.darksky.net/forecast/'
        self.berlin_latitude = '52.520008'
        self.berlin_longitude = '13.404954'
        self.exclude = '?exclude=minutely,hourly,currently,flags,alerts'
    
    def get_key(self):
        with open(self.path) as f:
            key = json.load(f)
            self.api_key = key['api_key']
            return self.api_key
    
    def get_weather(self, day):
        api_key = self.get_key()
        date = day+'T12:00:00'
        request_string = '{}{}/{},{},{}{}'.format(self.url, api_key, self.berlin_latitude, self.berlin_longitude, date,self.exclude)
        response = requests.get(request_string)
        self.weather = response.json()['daily']['data'][0]['icon']
        return self.weather
    
    def get_weather_for_all_days(self, dates):
        self.weather_df = pd.DataFrame(dates, columns = ['Date'])
        self.weather_df['Weather'] = ''
        for i in range(len(self.weather_df.Date)):
            self.weather_df['Weather'][i]= self.get_weather(self.weather_df['Date'][i])
        return self.weather_df
    
    def get_rain_win_percentage(self, dataset):
        final_df = pd.merge(dataset, self.weather_df, on='Date')
        final_df = final_df[final_df['Weather'] == 'rain'].reset_index()
        teams = set(final_df['HomeTeam'])
        rain_dict = {}
        for team in teams:
            total = 0
            wins = 0
            for i in range(len(final_df)):
                if (final_df['HomeTeam'][i] == team):
                    total += 1
                    if final_df['FTR'][i] == 'H':
                        wins += 1
                elif final_df['AwayTeam'][i] == team:
                    total +=1
                    if final_df['FTR'][i] == 'A':
                        wins += 1
            win_percent = round((wins/total)*100,2)
            rain_dict.update({team: win_percent})
        rain_win_df = pd.DataFrame.from_dict(rain_dict, orient='index').reset_index()
        rain_win_df.columns = ['TeamName', 'RainWinPercentage']
        return rain_win_df


In [11]:
weather = WeatherGetter()

In [12]:
weather_df = weather.get_weather_for_all_days(Matches_2011_df.Date.unique())
weather_df.head()

,Date,Weather
0,2012-03-31,rain
1,2011-12-11,partly-cloudy-day
2,2011-08-13,partly-cloudy-day
3,2011-11-27,partly-cloudy-day
4,2012-02-18,partly-cloudy-day


In [13]:
rain_win_df = weather.get_rain_win_percentage(Matches_2011_df)
rain_win_df.head()

,TeamName,RainWinPercentage
0,Aston Villa,16.67
1,Liverpool,40.00
2,Ein Frankfurt,33.33
3,Hamburg,15.38
4,Hoffenheim,20.00


# Getting the Wins and Loses for Each Team with SQL

In [14]:
# Create a dataframe that shows the number of wins, draws and loses for each team during the 2011 season

cur.execute('''SELECT HomeTeam AS TeamName, (HomeGoalCount + AwayGoalCount) AS TotalGoals, (WinsH+WinsA) AS Wins, (DrawsH+DrawsA) AS Draws, (LosesH+LosesA) AS Loses
FROM (
SELECT HomeTeam, 
SUM(FTHG) AS HomeGoalCount,
SUM(CASE FTR
    WHEN 'H' THEN 1
    WHEN 'D' THEN 0
    WHEN 'A' THEN 0
END) AS WinsH,
SUM(CASE FTR
    WHEN 'H' THEN 0
    WHEN 'D' THEN 1
    WHEN 'A' THEN 0
END) AS DrawsH,
SUM (CASE FTR
    WHEN 'H' THEN 0
    WHEN 'D' THEN 0
    WHEN 'A' THEN 1
END) AS LosesH
FROM Matches
WHERE Season = '2011'
GROUP BY HomeTeam) AS t1
JOIN
(SELECT AwayTeam,
SUM(FTAG) As AwayGoalCount,
SUM(CASE FTR
    WHEN 'H' THEN 0
    WHEN 'D' THEN 0
    WHEN 'A' THEN 1
END) AS WinsA,
SUM(CASE FTR
    WHEN 'H' THEN 0
    WHEN 'D' THEN 1
    WHEN 'A' THEN 0
END) AS DrawsA,
SUM (CASE FTR
    WHEN 'H' THEN 1
    WHEN 'D' THEN 0
    WHEN 'A' THEN 0
END) AS LosesA
FROM Matches
WHERE Season = '2011'
GROUP BY AwayTeam) AS t2
ON
t1.HomeTeam = t2.AwayTeam;''')
Stats_df = pd.DataFrame(cur.fetchall())
Stats_df.columns = [x[0] for x in cur.description]
Stats_df.head()

,TeamName,TotalGoals,Wins,Draws,Loses
0,Aachen,30,6,13,15
1,Arsenal,74,21,7,10
2,Aston Villa,37,7,17,14
3,Augsburg,36,8,14,12
4,Bayern Munich,77,23,4,7


In [15]:
# Merge rain_win_df and Stats_df

final_df = pd.merge(Stats_df, rain_win_df, on='TeamName')
final_df.head()

,TeamName,TotalGoals,Wins,Draws,Loses,RainWinPercentage
0,Aachen,30,6,13,15,23.08
1,Arsenal,74,21,7,10,57.14
2,Aston Villa,37,7,17,14,16.67
3,Augsburg,36,8,14,12,36.36
4,Bayern Munich,77,23,4,7,72.73


In [72]:
import gridfs

In [83]:
# Function to create histogram

def create_histogram(ind):
    data = final_df.iloc[ind].to_dict()
    x_values = list(data.keys())[2:5]
    y_values = list(data.values())[2:5]
    fig = plt.figure(figsize=(5,5))
    plt.title(list(data.values())[0] + ' 2011 Season Stats')
    plt.bar(x_values, y_values)
    plt.savefig('Histograms/' + list(data.values())[0] + '.png', bbox_inches='tight')
    plt.close(fig)
    

In [ ]:
client=MongoClient()
db=client.mytest
data=open(“image.png”,”rb”)
fs=gridfs.GridFS(db)
thedata=data.read()
stored=fs.put(thedata,filename=”inmongoimage”)

To get it back in the same code you call:
out=fs.get(stored).read()

In [64]:
create_histogram(0)

# MongoDB

In [117]:
from PIL import Image
from bson import Binary
import io

#img = Image.open('test.jpg')

#imgByteArr = io.BytesIO()
#img.save(imgByteArr, format='PNG')
#imgByteArr = imgByteArr.getvalue()

In [133]:
class MongoHandler:
    def __init__(self):
        self.myclient = pymongo.MongoClient()
        self.database = self.myclient['soccer_stats_database']
        self.collection = self.database['soccer_stats_collection']
        
    def reformat_png(self, png_file):
        data=open(png_file, "rb")        
        fs=gridfs.GridFS(self.database)
        thedata=data.read()
        stored=fs.put(thedata, filename="mongoimage")
        return fs.get(stored).read()
    
    def format_data(self, ind):
        self.stats_dict = {
            "team" : final_df.TeamName[i],
            "histogram" : self.reformat_png('Histograms/' + final_df.TeamName[i] + '.png')  
        }
        return self.stats_dict
    
    def insert_record(self, ind):
        #data = soccer_Team_Data_in_dict_Form #{'name': 'John Doe', 'address': '123 elm street', 'age': 28}
        stats_dict = self.format_data(ind)
        return self.collection.insert_one(stats_dict)
    
    def view_mongo_collection(self):
        query = self.collection.find({})
        for x in query:
            print(x)
    
    def delete_collection(self):
        self.collection.delete_many({})

In [134]:
mongo = MongoHandler()

In [135]:
for i in range(3):
    create_histogram(i)
    
    mongo.insert_record(i)

In [125]:
#for i in range(len(final_df)):
for i in range(3):
    create_histogram(i)
    stats_dict = {
        "team" : final_df.TeamName[i],
        "total_goals" : float(final_df.TotalGoals[i]),
        "total_wins" : float(final_df.Wins[i]),
        "histogram" : Image.open('Histograms/' + final_df.TeamName[i] + '.png').show(),
        "rain_win_percentage" : final_df.RainWinPercentage[i]
    }
    mongo.insert_record(stats_dict)

TypeError: data must be an instance of bytes

In [136]:
mongo.view_mongo_collection()

{'_id': ObjectId('5db0d9202a3a10c43e9cd102'), 'team': 'Aachen', 'histogram': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01<\x00\x00\x01?\x08\x06\x00\x00\x00\xdb\x91\x7f\xa8\x00\x00\x00\x04sBIT\x08\x08\x08\x08|\x08d\x88\x00\x00\x00\tpHYs\x00\x00\x0b\x12\x00\x00\x0b\x12\x01\xd2\xdd~\xfc\x00\x00\x009tEXtSoftware\x00matplotlib version 3.0.3, http://matplotlib.org/\x9d\x0b\xab\xa3\x00\x00\x1d6IDATx\x9c\xed\xdd}XTu\xfe\xff\xf1\x17\x82\xf7\xc0\xc8\xe4\x00\xc2`\xe4M\x82r\x17B\n\x06Y$\xb8j\xb8[\xd6fT\x10\xb6\xacWV\xa6\xad\x9b\x9b\xbb_\xef*Q3\xed\xee\xcaf\x97\x14-\xb5\x9b-m\xb5,5KM\xd6\x9c\nMQtU\x08\xd0r0@\x10\x81\x01\xde\xbf?\xfcy6\x16\x95\x9c9#\xd9\xe7\xf5\xb8.\xae\xcb9\xe7\xcc\x9c\xf79\xda\x9333\x0c\xb9\x89\x88\x80\x88H\x01\x1d\xda{\x00"\xa2+\x85\xc1#"e0xD\xa4\x0c\x06\x8f\x88\x94\xc1\xe0\x11\x912\x18<"R\x06\x83w\x15\xca\xc8\xc8\xc0_\xff\xfa\xd7\xf6\x1e\x83\xe8\xaa\xc3\xe0]\x01\xc3\x87\x0f\x87\x8f\x8f\x0f\xea\xeb\xeb\xdb{\x94\xcbr\xe8\xd0!\x8c\x1d;\x16&\x93\tF\xa3\x11)))(,,l\xb1\xcd\xe2\xc

In [131]:
mongo.delete_collection()

In [132]:
mongo.view_mongo_collection()

TypeError: object of type 'NoneType' has no len()